## Purview Credential from the Purview API.

In [16]:
from azure.purview.scanning import PurviewScanningClient
from azure.identity import ClientSecretCredential 
from azure.purview.administration.account import PurviewAccountClient
from dotenv import load_dotenv

import os

load_dotenv()

client_id = os.environ.get("AZURE_CLIENT_ID")
client_secret = os.environ.get("AZURE_CLIENT_SECRET")
tenant_id = os.environ.get("AZURE_TENANT_ID")
purview_endpoint = os.environ.get("PURVIEW_ENDPOINT")
purview_scan_endpoint = os.environ.get("PURVIEW_SCAN_ENDPOINT")

def get_credentials():
	credentials = ClientSecretCredential(client_id=client_id, client_secret=client_secret, tenant_id=tenant_id)
	return credentials

def get_purview_client():
	credentials = get_credentials()
	client = PurviewScanningClient(endpoint=purview_scan_endpoint, credential=credentials, logging_enable=True)  
	return client

def get_admin_client():
	credentials = get_credentials()
	client = PurviewAccountClient(endpoint=purview_endpoint, credential=credentials, logging_enable=True)
	return client

In [17]:
credential = get_credentials()

In [22]:
token_obj = credential.get_token('https://purview.azure.net/.default')

#### If your Tenant is restricted to issue a token due to security issue, Please try to get the token in VPN. 
#### Without VPN, you may get the error "AADSTS500011: The resource principal named https://purview.azure.net was not found in the tenant named ..."

In [ ]:
# Set up the headers for the Purview Search API request
import requests
access_token = token_obj.token

headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Define the search query (customize as needed)
search_url = f"https://{purview_endpoint}/catalog/api/atlas/v2/glossary/search"

# Make the search API request
response = requests.get(search_url, headers=headers)

# Check and print the response
if response.status_code == 200:
    search_results = response.json().get('entities', [])
    print(f"Search returned {len(search_results)} results:")
else:
    print(f"Search API call failed: {response.text}")

Search API call failed: {"requestId":"ea4924cc-90a1-4725-a48d-d411a4910c5e","errorCode":"ATLAS-404-00-005","errorMessage":"Given instance guid search is invalid/not found"}


## Device code flow (MFA) or Acquire a token interactively

In [ ]:
import msal
import requests

# Azure AD App Registration settings
# Azure AD and Purview settings
client_id = os.environ.get("AZURE_CLIENT_ID")
tenant_id = os.environ.get("AZURE_TENANT_ID")
purview_endpoint = os.environ.get("PURVIEW_ENDPOINT")


# Authentication authority and scope for Purview API
authority = f"https://login.microsoftonline.com/{tenant_id}"
scope = ["https://purview.azure.net/.default"]  # API scope for Azure Purview

# Initialize MSAL public client application
app = msal.PublicClientApplication(client_id, authority=authority)

# Initiate the device code flow for interactive authentication
flow = app.initiate_device_flow(scopes=scope)
if "user_code" not in flow:
    raise Exception("Device flow initiation failed.")

# Display the code and URL for the user to authenticate
print(f"Go to {flow['verification_uri']} and enter the code: {flow['user_code']}")

# Wait for user to complete authentication and MFA
token_result = app.acquire_token_by_device_flow(flow)
if "access_token" not in token_result:
    raise Exception(f"Authentication failed: {token_result.get('error_description')}")

access_token = token_result['access_token']

# Set up the headers for the Purview Search API request
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Define the search query (customize as needed)
search_url = f"{purview_endpoint}/catalog/api/atlas/v2/search/advanced"
search_body = {
    "keywords": "Product",  # Replace with your search term
    "limit": 10  # Limit the number of search results
}

# Make the search API request
response = requests.post(search_url, json=search_body, headers=headers)

# Check and print the response
if response.status_code == 200:
    search_results = response.json().get('entities', [])
    for asset in search_results:
        print(f"Name: {asset['attributes']['name']}, Type: {asset['typeName']}, GUID: {asset['guid']}")
else:
    print(f"Search API call failed: {response.text}")

In [ ]:
import msal

# Azure AD App Registration settings
# Azure AD and Purview settings
client_id = os.environ.get("AZURE_CLIENT_ID")
tenant_id = os.environ.get("AZURE_TENANT_ID")
purview_endpoint = os.environ.get("PURVIEW_ENDPOINT")


# Authentication authority and scope for Purview API
authority = f"https://login.microsoftonline.com/{tenant_id}"
scope = ["https://purview.azure.net/.default"]  # API scope for Azure Purview

# Initialize MSAL public client application
app = msal.PublicClientApplication(client_id, authority=authority)

result = app.acquire_token_interactive(scopes=scope)

if "access_token" in result:
    print("Access token acquired:")
    print(result["access_token"])
else:
    print("Failed to acquire token:")
    print(result.get("error_description"))
